In [1]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('pokemon.db')
cursor = conn.cursor()

# Lista de tablas que quieres procesar
tablas_a_procesar = ['quinta_generacion', 'sexta_generacion', 'septima_generacion', 'octava_generacion']


# Función para obtener los nombres y tipos de columnas de una tabla, excluyendo la columna id_generacion
def obtener_columnas(cursor, tabla):  # Se relaciona con paso 5
    cursor.execute(f"PRAGMA table_info({tabla})")
    columns = cursor.fetchall()  # Regresa cada col como una tupla. Cada tupla representa una fila de la información de una columna de la tabla
# y contiene detalles como el nombre de la columna, el tipo de datos, si es nula, etc.
# Una tupla típica podría ser ('id_generacion', 'INTEGER', 1, 0, None, 1)
    column_names = [col[1] for col in columns if col[1] != 'id_generacion']
    column_types = [col[2] for col in columns if col[1] != 'id_generacion']
    return column_names, column_types  # Regresa dos listas


# Definir los tipos de columnas en la nueva tabla
def get_column_definition(name, col_type):
    # Considerar los tipos de datos y definir TEXT para texto con dígitos o caracteres especiales
    if col_type in ('INTEGER', 'REAL'):
        return f'{name} {col_type}'
    else:
        return f'{name} TEXT'


# Procesar cada tabla en la lista
for tabla in tablas_a_procesar:
    # (copiar nombre + sufijo '_copia')
    copia_tabla = f'{tabla}_copia'

    # Obtener nombres y tipos de columnas. Llamado a func obtener_columnas
    column_names, column_types = obtener_columnas(cursor, tabla)

    # Crear las definiciones de columnas. Define cada col en SQL. Extrae el nombre y tipo de dato
    column_definitions = [get_column_definition(name, col_type) for name, col_type in zip(column_names, column_types)]

    # Eliminar la tabla copia si ya existe
    drop_table_sql = f'DROP TABLE IF EXISTS "{copia_tabla}";'
    try:
        cursor.execute(drop_table_sql)
    except sqlite3.OperationalError as e:
        print(f'Error al eliminar la tabla {copia_tabla}: {e}')
        conn.close()
        raise

    # Crear la nueva tabla con columna autoincremental
    create_table_sql = f'''
    CREATE TABLE {copia_tabla}(
        id_generacion INTEGER PRIMARY KEY AUTOINCREMENT,
        {', '.join(column_definitions)}
    );
    '''

    print(f"SQL de creación de tabla para {copia_tabla}:")
    print(create_table_sql)  # Imprimir el SQL para depuración

    try:
        cursor.execute(create_table_sql)
    except sqlite3.OperationalError as e:
        print(f'Error al crear la tabla {copia_tabla}: {e}')
        conn.close()
        raise

    # Copiar los datos desde la tabla original
    columns_str = ', '.join(column_names)
    insert_data_sql = f'''
    INSERT INTO {copia_tabla} ({columns_str})
    SELECT {columns_str}
    FROM {tabla};
    '''

    print(f"SQL de inserción de datos para {copia_tabla}:")
    print(insert_data_sql)  # Imprimir el SQL para depuración

    try:
        cursor.execute(insert_data_sql)
    except sqlite3.OperationalError as e:
        print(f'Error al insertar datos en {copia_tabla}: {e}')

    print(f'*************Tabla {tabla} exportada con éxito*************')

# Confirmar y cerrar conexión
conn.commit()
conn.close()


SQL de creación de tabla para quinta_generacion_copia:

    CREATE TABLE quinta_generacion_copia(
        id_generacion INTEGER PRIMARY KEY AUTOINCREMENT,
        id_global INTEGER, nombre TEXT, tipo_1 TEXT, tipo_2 TEXT, japones TEXT, romanizado TEXT, rom_oficial TEXT, g_huevo_1 TEXT, g_huevo_2 TEXT
    );
    
SQL de inserción de datos para quinta_generacion_copia:

    INSERT INTO quinta_generacion_copia (id_global, nombre, tipo_1, tipo_2, japones, romanizado, rom_oficial, g_huevo_1, g_huevo_2)
    SELECT id_global, nombre, tipo_1, tipo_2, japones, romanizado, rom_oficial, g_huevo_1, g_huevo_2
    FROM quinta_generacion;
    
*************Tabla quinta_generacion exportada con éxito*************
SQL de creación de tabla para sexta_generacion_copia:

    CREATE TABLE sexta_generacion_copia(
        id_generacion INTEGER PRIMARY KEY AUTOINCREMENT,
        id_global INTEGER, nombre TEXT, tipo_1 TEXT, tipo_2 TEXT, japones TEXT, romanizado TEXT, rom_oficial TEXT, g_huevo_1 TEXT, g_huevo_2 